In [2]:
import numpy as np
from sklearn import preprocessing
from keras import utils
import numpy.matlib

SD = np.loadtxt("SD.txt", dtype=float, delimiter=" ")
SM = np.loadtxt("SM.txt", dtype=float, delimiter=" ")
A = np.loadtxt("interaction.txt", dtype=int, delimiter=" ")
interacation = np.transpose(A)


In [3]:
from tensorflow.keras.utils import to_categorical


R_B = np.matlib.repmat(interacation, 495, 1)
sm = np.repeat(SM, repeats=383, axis=0)
train_m = np.concatenate((sm, R_B), axis=1)  # (189585,990)

R_A = np.repeat(A, repeats=383, axis=0)
sd = np.matlib.repmat(SD, 495, 1)
train_s = np.concatenate((R_A, sd), axis=1)  # (189585,766)
label = A.reshape((189585, 1))

# Initialize LabelEncoder and fit it to the labels
encoder = preprocessing.LabelEncoder()
encoder.fit(label)

# Transform the labels to integer values
ys = encoder.transform(label).astype(np.int32)

# Convert to categorical (one-hot encoding) if needed
ys = to_categorical(ys)

nm = np.arange(len(ys))
ys = ys[nm]



c:\Users\shahe\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\shahe\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [4]:
print(train_m) 
print(train_s) 

[[1.       0.249269 0.093465 ... 0.       0.       0.      ]
 [1.       0.249269 0.093465 ... 0.       0.       0.      ]
 [1.       0.249269 0.093465 ... 0.       0.       0.      ]
 ...
 [0.077888 0.064907 0.161507 ... 0.       0.       0.      ]
 [0.077888 0.064907 0.161507 ... 0.       0.       0.      ]
 [0.077888 0.064907 0.161507 ... 0.       0.       0.      ]]
[[1.       0.       0.       ... 0.86843  0.49394  0.568774]
 [1.       0.       0.       ... 0.754171 0.264119 0.568774]
 [1.       0.       0.       ... 0.530038 0.203341 0.61034 ]
 ...
 [0.       0.       0.       ... 1.       0.568774 0.654945]
 [0.       0.       0.       ... 0.568774 1.       0.49394 ]
 [0.       0.       0.       ... 0.654945 0.49394  1.      ]]


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

encoding_dim = 64
input_mdata = layers.Input(shape=(990,))

encoded = layers.Dense(350, activation='relu')(input_mdata)
encoded = layers.Dense(250, activation='relu')(encoded)
encoded = layers.Dense(100, activation='relu')(encoded)
miRNA_encoder_output = layers.Dense(encoding_dim)(encoded)

decoded = layers.Dense(100, activation='relu')(miRNA_encoder_output)
decoded = layers.Dense(250, activation='relu')(decoded)
decoded = layers.Dense(350, activation='relu')(decoded)
decoded = layers.Dense(990, activation='sigmoid')(decoded)  # or 'tanh' if data is scaled accordingly

autoencoder = models.Model(input_mdata, decoded)
encoder = models.Model(input_mdata, miRNA_encoder_output)

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(train_m, train_m, epochs=15, batch_size=100, shuffle=True)
miRNA_encoded_datas = encoder.predict(train_m)


Epoch 1/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 30s 13ms/step - loss: 0.0159
Epoch 2/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 24s 12ms/step - loss: 0.0036
Epoch 3/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - loss: 0.0029
Epoch 4/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - loss: 0.0025
Epoch 5/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 24s 12ms/step - loss: 0.0027
Epoch 6/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - loss: 0.0025
Epoch 7/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - loss: 0.0020
Epoch 8/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - loss: 0.0019
Epoch 9/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - loss: 0.0019
Epoch 10/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - loss: 0.0018
Epoch 11/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - loss: 0.0018
Epoch 12/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 28s 14ms/step - loss: 0.0018
Epoch 13/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - loss: 0.0019
Epoch 14/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - loss: 0.0019
E

In [ ]:
encoding_dim = 64
input_ddata = layers.Input(shape=(766,))

encoded = layers.Dense(350, activation='relu')(input_ddata)
encoded = layers.Dense(250, activation='relu')(encoded)
encoded = layers.Dense(100, activation='relu')(encoded)
disease_encoder_output = layers.Dense(encoding_dim)(encoded)

decoded = layers.Dense(100, activation='relu')(disease_encoder_output)
decoded = layers.Dense(250, activation='relu')(decoded)
decoded = layers.Dense(350, activation='relu')(decoded)
decoded = layers.Dense(766, activation='sigmoid')(decoded)

autoencoder = models.Model(input_ddata, decoded)
encoder = models.Model(input_ddata, disease_encoder_output)

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(train_s, train_s, epochs=15, batch_size=100, shuffle=True)
disease_encoded_datas = encoder.predict(train_s)

Epoch 1/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - loss: 0.0185
Epoch 2/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - loss: 0.0048
Epoch 3/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - loss: 0.0037
Epoch 4/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - loss: 0.0034
Epoch 5/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 34s 18ms/step - loss: 0.0033
Epoch 6/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step - loss: 0.0028
Epoch 7/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - loss: 0.0026
Epoch 8/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - loss: 0.0026
Epoch 9/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - loss: 0.0024
Epoch 10/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - loss: 0.0024
Epoch 11/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - loss: 0.0023
Epoch 12/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 26s 14ms/step - loss: 0.0023
Epoch 13/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - loss: 0.0023
Epoch 14/15
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - loss: 0.0022
Ep

In [ ]:
autoencoder.save("autoencoder_disease_model.h5")  # Save the entire autoencoder model
encoder.save("encoder_disease_model.h5")          # Save only the encoder model

In [ ]:
# Load the autoencoder model
loaded_autoencoder = tf.keras.models.load_model("autoencoder_disease_model.h5")

# Load the encoder model
loaded_encoder = tf.keras.models.load_model("encoder_disease_model.h5")

# Use the loaded encoder for predictions
miRNA_encoded_datas = loaded_encoder.predict(train_m)

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, roc_auc_score, confusion_matrix, matthews_corrcoef
import matplotlib.pyplot as plt

X = np.hstack((miRNA_encoded_datas, disease_encoded_datas))
y = A.flatten()

# Initialize lists to store evaluation metrics
accuracy_results, sensitivity_results, specificity_results, precision_results, mcc_results = [], [], [], [], []
y_true, y_scores = [], []
confusion_matrices = []

# Define KFold cross-validation (5-fold or 10-fold)
kf = KFold(n_splits=3, shuffle=True, random_state=42)  # Use n_splits=10 for 10-fold CV

# Cross-validation loop
for train_index, val_index in kf.split(X):
    x_train, x_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Initialize RandomForestClassifier with class weights to handle imbalance
    clf = RandomForestClassifier(n_estimators=10, max_depth=20,min_samples_split=5,min_samples_leaf=3,random_state=42,class_weight="balanced")
    clf.fit(x_train, y_train)

    # Get predictions and probabilities for evaluation
    y_pred = clf.predict(x_val)
    y_pred_proba = clf.predict_proba(x_val)[:, 1]  # Probabilities for the positive class

    # Store true labels and predicted probabilities for ROC-AUC calculation
    y_true.extend(y_val)
    y_scores.extend(y_pred_proba)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)  # Sensitivity
    mcc = matthews_corrcoef(y_val, y_pred)
    
    # Calculate specificity using the confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Append each fold's metrics to results lists
    accuracy_results.append(accuracy)
    sensitivity_results.append(recall)  # Sensitivity
    specificity_results.append(specificity)
    precision_results.append(precision)
    mcc_results.append(mcc)
    
    # Calculate confusion matrix for the fold and store it
    cm = confusion_matrix(y_val, y_pred)
    confusion_matrices.append(cm)


# Calculate average metrics across all folds
avg_accuracy = sum(accuracy_results) / len(accuracy_results)
avg_sensitivity = sum(sensitivity_results) / len(sensitivity_results)
avg_specificity = sum(specificity_results) / len(specificity_results)
avg_precision = sum(precision_results) / len(precision_results)
avg_mcc = sum(mcc_results) / len(mcc_results)
roc_auc = roc_auc_score(y_true, y_scores)  # Overall ROC-AUC score

final_cm = confusion_matrices[-1]
print(f"Confusion Matrix:\n{final_cm}\n")

# Display results
print(f"Average Accuracy (Acc): {avg_accuracy:.4f}")
print(f"Average Sensitivity (Sen): {avg_sensitivity:.4f}")
print(f"Average Specificity (Spe): {avg_specificity:.4f}")
print(f"Average Precision (Pre): {avg_precision:.4f}")
print(f"Average Matthews Correlation Coefficient (Mcc): {avg_mcc:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")

fpr, tpr, thresholds = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='blue', label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.savefig('roc_curve.png') 

plt.show()

NameError: name 'miRNA_encoded_datas' is not defined

In [ ]:
# import numpy as np
# from sklearn.model_selection import KFold
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
# import matplotlib.pyplot as plt

# X = np.hstack((miRNA_encoded_datas, disease_encoded_datas))
# y = ys.flatten()



# num_folds = 5
# kf = KFold(n_splits=num_folds, shuffle=True)

# accuracy_results = []
# precision_results = []
# recall_results = []
# f1_results = []
# y_true = []
# y_scores = []


In [ ]:
# #avg_sensitivity mcc 


# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import KFold

# # Initialize lists to store evaluation metrics
# accuracy_results, sensitivity_results, specificity_results, precision_results, mcc_results = [], [], [], [], []
# y_true, y_scores = [], []
# confusion_matrices = []
# threshold = 0.4

# # Define KFold cross-validation (5-fold or 10-fold)
# kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Use n_splits=10 for 10-fold CV

# # Cross-validation loop
# for train_index, val_index in kf.split(X):
#     x_train, x_val = X[train_index], X[val_index]
#     y_train, y_val = y[train_index], y[val_index]

#     # Initialize RandomForestClassifier with class weights to handle imbalance
#     clf = RandomForestClassifier(n_estimators=300, max_depth=25,min_samples_split=20,min_samples_leaf=10,class_weight="balanced")
#     clf.fit(x_train, y_train)

#     # Get predictions and probabilities for evaluation
#     y_pred = clf.predict(x_val)
#     y_pred_proba = clf.predict_proba(x_val)[:, 1]  # Probabilities for the positive class
#     y_pred_custom = (y_pred_proba >= threshold).astype(int)

#     # Store true labels and predicted probabilities for ROC-AUC calculation
#     y_true.extend(y_val)
#     y_scores.extend(y_pred_proba)

#     # Calculate evaluation metrics
#     accuracy = accuracy_score(y_val, y_pred)
#     precision = precision_score(y_val, y_pred)
#     recall = recall_score(y_val, y_pred)  # Sensitivity
#     mcc = matthews_corrcoef(y_val, y_pred)
    
#     # Calculate specificity using the confusion matrix
#     tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
#     specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

#     # Append each fold's metrics to results lists
#     accuracy_results.append(accuracy)
#     sensitivity_results.append(recall)  # Sensitivity
#     specificity_results.append(specificity)
#     precision_results.append(precision)
#     mcc_results.append(mcc)
    
#     # Calculate confusion matrix for the fold and store it
#     cm = confusion_matrix(y_val, y_pred)
#     confusion_matrices.append(cm)

# # Calculate average metrics across all folds
# avg_accuracy = sum(accuracy_results) / len(accuracy_results)
# avg_sensitivity = sum(sensitivity_results) / len(sensitivity_results)
# avg_specificity = sum(specificity_results) / len(specificity_results)
# avg_precision = sum(precision_results) / len(precision_results)
# avg_mcc = sum(mcc_results) / len(mcc_results)
# roc_auc = roc_auc_score(y_true, y_scores)  # Overall ROC-AUC score

# # Display results
# print(f"Average Accuracy (Acc): {avg_accuracy:.4f}")
# print(f"Average Sensitivity (Sen): {avg_sensitivity:.4f}")
# print(f"Average Specificity (Spe): {avg_specificity:.4f}")
# print(f"Average Precision (Pre): {avg_precision:.4f}")
# print(f"Average Matthews Correlation Coefficient (Mcc): {avg_mcc:.4f}")
# print(f"ROC-AUC Score: {roc_auc:.4f}")

# # Print confusion matrices for each fold
# for i, cm in enumerate(confusion_matrices):
#     print(f"Confusion Matrix for Fold {i + 1}:\n{cm}\n")


In [ ]:
# fpr, tpr, thresholds = roc_curve(y_true, y_scores)
# roc_auc = auc(fpr, tpr)

# # Plot ROC curve
# plt.figure()
# plt.plot(fpr, tpr, color='blue', label='ROC curve (AUC = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='red', linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.0])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc='lower right')
# plt.savefig('roc_curve.png')

# plt.show()


In [ ]:
#balanced in randomforestclassifier

# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# from sklearn.ensemble import RandomForestClassifier

# # Initialize lists to store evaluation metrics
# accuracy_results, precision_results, recall_results, f1_results = [], [], [], []
# y_true, y_scores = [], []

# # KFold cross-validation
# for train_index, val_index in kf.split(X):
#     x_train, x_val = X[train_index], X[val_index]
#     y_train, y_val = y[train_index], y[val_index]

#     # Add class weights to the classifier
#     clf = RandomForestClassifier(n_estimators=300, class_weight="balanced")
#     clf.fit(x_train, y_train)

#     # Get predictions and probabilities
#     y_pred = clf.predict(x_val)
#     y_pred_proba = clf.predict_proba(x_val)[:, 1]  # Probabilities for the positive class

#     # Store true labels and predicted probabilities for ROC calculation
#     y_true.extend(y_val)
#     y_scores.extend(y_pred_proba)

#     # Calculate metrics at default threshold (0.5)
#     accuracy = accuracy_score(y_val, y_pred)
#     precision = precision_score(y_val, y_pred)
#     recall = recall_score(y_val, y_pred)
#     f1 = f1_score(y_val, y_pred)

#     # Append metrics to the results lists
#     accuracy_results.append(accuracy)
#     precision_results.append(precision)
#     recall_results.append(recall)
#     f1_results.append(f1)

# # Calculate average metrics across all folds
# avg_accuracy = sum(accuracy_results) / len(accuracy_results)
# avg_precision = sum(precision_results) / len(precision_results)
# avg_recall = sum(recall_results) / len(recall_results)
# avg_f1 = sum(f1_results) / len(f1_results)
# roc_auc = roc_auc_score(y_true, y_scores)  # Overall ROC-AUC score

# print(f"Average Accuracy: {avg_accuracy:.4f}")
# print(f"Average Precision: {avg_precision:.4f}")
# print(f"Average Recall: {avg_recall:.4f}")
# print(f"Average F1 Score: {avg_f1:.4f}")
# print(f"ROC-AUC Score: {roc_auc:.4f}")


In [ ]:

# for train_index, val_index in kf.split(X):
#     x_train, x_val = X[train_index], X[val_index]
#     y_train, y_val = y[train_index], y[val_index]

#     clf = RandomForestClassifier(n_estimators=100)
#     clf.fit(x_train, y_train)

#     y_pred = clf.predict(x_val)
#     y_pred_proba = clf.predict_proba(x_val)[:, 1]  # Get the probabilities for the positive class

#     # Store true labels and predicted probabilities for ROC calculation
#     y_true.extend(y_val)
#     y_scores.extend(y_pred_proba)

#     accuracy = accuracy_score(y_val, y_pred)
#     precision = precision_score(y_val, y_pred)
#     recall = recall_score(y_val, y_pred)
#     f1 = f1_score(y_val, y_pred)

#     accuracy_results.append(accuracy)
#     precision_results.append(precision)
#     recall_results.append(recall)
#     f1_results.append(f1)


In [ ]:

# average_accuracy = np.mean(accuracy_results)
# average_precision = np.mean(precision_results)
# average_recall = np.mean(recall_results)
# average_f1 = np.mean(f1_results)

# print(f'Average Accuracy: {average_accuracy:.4f}')
# print(f'Average Precision: {average_precision:.4f}')
# print(f'Average Recall: {average_recall:.4f}')
# print(f'Average F1 Score: {average_f1:.4f}')

# # Calculate ROC curve and AUC
# fpr, tpr, thresholds = roc_curve(y_true, y_scores)
# roc_auc = auc(fpr, tpr)

# # Plot ROC curve
# plt.figure()
# plt.plot(fpr, tpr, color='blue', label='ROC curve (AUC = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='red', linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.0])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc='lower right')
# plt.savefig('roc_curve.png')

# plt.show()


In [ ]:
# import numpy as np
# from sklearn.model_selection import KFold
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
# import matplotlib.pyplot as plt
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# from sklearn.ensemble import RandomForestClassifier


# X = np.hstack((train_m, train_s))
# y = ys.flatten()

# num_folds = 5
# kf = KFold(n_splits=num_folds, shuffle=True)

# accuracy_results = []
# precision_results = []
# recall_results = []
# f1_results = []
# y_true = []
# y_scores = []


# #balanced in randomforestclassifier


# # Initialize lists to store evaluation metrics
# accuracy_results, precision_results, recall_results, f1_results = [], [], [], []
# y_true, y_scores = [], []

# # KFold cross-validation
# for train_index, val_index in kf.split(X):
#     x_train, x_val = X[train_index], X[val_index]
#     y_train, y_val = y[train_index], y[val_index]

#     # Add class weights to the classifier
#     clf = RandomForestClassifier(n_estimators=100, class_weight="balanced")
#     clf.fit(x_train, y_train)

#     # Get predictions and probabilities
#     y_pred = clf.predict(x_val)
#     y_pred_proba = clf.predict_proba(x_val)[:, 1]  # Probabilities for the positive class

#     # Store true labels and predicted probabilities for ROC calculation
#     y_true.extend(y_val)
#     y_scores.extend(y_pred_proba)

#     # Calculate metrics at default threshold (0.5)
#     accuracy = accuracy_score(y_val, y_pred)
#     precision = precision_score(y_val, y_pred)
#     recall = recall_score(y_val, y_pred)
#     f1 = f1_score(y_val, y_pred)

#     # Append metrics to the results lists
#     accuracy_results.append(accuracy)
#     precision_results.append(precision)
#     recall_results.append(recall)
#     f1_results.append(f1)

# # Calculate average metrics across all folds
# avg_accuracy = sum(accuracy_results) / len(accuracy_results)
# avg_precision = sum(precision_results) / len(precision_results)
# avg_recall = sum(recall_results) / len(recall_results)
# avg_f1 = sum(f1_results) / len(f1_results)
# roc_auc = roc_auc_score(y_true, y_scores)  # Overall ROC-AUC score

# print(f"Average Accuracy: {avg_accuracy:.4f}")
# print(f"Average Precision: {avg_precision:.4f}")
# print(f"Average Recall: {avg_recall:.4f}")
# print(f"Average F1 Score: {avg_f1:.4f}")
# print(f"ROC-AUC Score: {roc_auc:.4f}")

# # Calculate ROC curve and AUC
# fpr, tpr, thresholds = roc_curve(y_true, y_scores)
# roc_auc = auc(fpr, tpr)

# # Plot ROC curve
# plt.figure()
# plt.plot(fpr, tpr, color='blue', label='ROC curve (AUC = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='red', linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.0])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc='lower right')
# plt.savefig('roc_curve.png')

# plt.show()
